In [3]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from datetime import datetime

# Setup the Google Sheets API client
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name('jitta-intel-dev-bf2d51598b86.json', scope)
client = gspread.authorize(creds)

# Data string
data = """2014-10-01T00:00:00.000Z 0.03514797442727047
2015-10-01T00:00:00.000Z 0.08382339168799713
2016-10-01T00:00:00.000Z 0.2788915067066573
2017-10-01T00:00:00.000Z 0.035875500445650355
2018-10-01T00:00:00.000Z -0.08378928395743446
2019-10-01T00:00:00.000Z -0.10422462023501577
2020-10-01T00:00:00.000Z 0.6505359435764599
2021-10-01T00:00:00.000Z -0.15010136412876718
2022-10-01T00:00:00.000Z 0.19079751268557787
Error:
2014-10-02T00:00:00.000Z 0.02820418641725353
2015-10-02T00:00:00.000Z 0.07187181800815967
2016-10-02T00:00:00.000Z 0.2835171561441121"""

# Parsing the data
lines = data.split('\n')
data_entries = []

for line in lines:
    if line.strip() and not line.startswith("Error:"):
        date_str, value_str = line.split()
        date = datetime.strptime(date_str, "%Y-%m-%dT%H:%M:%S.%fZ")
        value = float(value_str)
        data_entries.append((date, value))

# Function to get or create a sheet by name
def get_or_create_sheet(sheet_name):
    try:
        sheet = client.open('USrby').worksheet(sheet_name)
    except gspread.exceptions.WorksheetNotFound:
        sheet = client.open('USrby').add_worksheet(title=sheet_name, rows="100", cols="2")
    return sheet

# Appending data to the respective sheets
for date, value in data_entries:
    sheet_name = f"{date.month}-{date.day}"
    sheet = get_or_create_sheet(sheet_name)
    sheet.append_row([date.isoformat(), value])
